In [428]:
# Курсовой проект для курса "Python для Data Science"

# Материалы к проекту (файлы):
# train.csv
# test.csv

# Задание:
# Используя данные из train.csv, построить
# модель для предсказания цен на недвижимость (квартиры).
# С помощью полученной модели предсказать
# цены для квартир из файла test.csv.

# Целевая переменная:
# Price

# Основная метрика:
# R2 - коэффициент детерминации (sklearn.metrics.r2_score)

# Вспомогательная метрика:
# MSE - средняя квадратичная ошибка (sklearn.metrics.mean_squared_error)

In [429]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [430]:
data = pd.read_csv('train.csv')

In [431]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [432]:
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())
data['Healthcare_1'] = data['Healthcare_1'].fillna(data['Healthcare_1'].mean())

In [433]:
data.loc[data['Rooms'] > 5]
data.loc[377, 'Rooms'] = 3
data.loc[1454, 'Rooms'] = 2
data.loc[2170, 'Rooms'] = 3
data.loc[8849, 'Rooms'] = 3

In [434]:
del data['HouseFloor']

In [435]:
data.loc[data['LifeSquare'] > data['Square'], 'Square'] = data['LifeSquare']

In [436]:
data.loc[(data['KitchenSquare'] == 0) & (data['Square'] > data['LifeSquare']), 'KitchenSquare'] = \
data['Square'] - data['LifeSquare']

In [437]:
data.loc[(data['Rooms'] < 1) & (data['LifeSquare'] > 35), 'Rooms'] = 3

In [438]:
data.loc[(data['Rooms'] < 1)] = 1

In [439]:
data.loc[(data['Square'] < 15) & (data['LifeSquare'] < 15) & (data['Rooms'] <= 1), 'Square'] = \
data.loc[data['Rooms'] <= 1, 'Square'].mean()

In [440]:
data.loc[(data['Square'] < 15) & (data['LifeSquare'] < 15) & (data['Rooms'] == 3), 'Square'] = \
data.loc[data['Rooms'] == 3, 'Square'].mean()

In [441]:
data.loc[(data['Rooms'] == 3) & (data['LifeSquare'] < 35), 'LifeSquare'] = \
data.loc[data['Rooms'] == 3, 'LifeSquare'].mean()

In [442]:
data.loc[(data['Rooms'] == 2) & (data['LifeSquare'] < 20), 'LifeSquare'] = \
data.loc[data['Rooms'] == 2, 'LifeSquare'].mean()

In [443]:
data.loc[(data['Rooms'] == 1) & (data['LifeSquare'] < 15), 'LifeSquare'] = \
data.loc[data['Rooms'] == 1, 'LifeSquare'].mean()

In [444]:
data.loc[(data['Price'] > data['Price'].mean()) & (data['Rooms'] == 1), 'Price'] = \
data.loc[data['Rooms'] == 1, 'Price'].mean()

In [445]:
data.loc[(data['Price'] > data['Price'].mean()) & (data['Rooms'] == 2), 'Price'] = \
data.loc[data['Rooms'] == 2, 'Price'].mean()

In [446]:
data.loc[(data['Price'] > data['Price'].mean()) & (data['Rooms'] == 3), 'Price'] = \
data.loc[data['Rooms'] == 3, 'Price'].mean()

In [447]:
data.drop(4328, inplace=True)

In [448]:
data.loc[(data['Rooms'] == 4) & (data['LifeSquare'] < 45), 'LifeSquare'] = \
data.loc[data['Rooms'] == 4, 'LifeSquare'].mean()
data.loc[(data['Rooms'] == 5) & (data['LifeSquare'] < 55), 'LifeSquare'] = \
data.loc[data['Rooms'] == 5, 'LifeSquare'].mean()

In [449]:
data.loc[(data['KitchenSquare'] > data['LifeSquare']) 
         & (data['KitchenSquare'] + data['LifeSquare'] == data['Square']), ['LifeSquare', 'KitchenSquare']] = \
data.loc[(data['KitchenSquare'] > data['LifeSquare']) 
         & (data['KitchenSquare'] + data['LifeSquare'] == data['Square']), ['KitchenSquare', 'LifeSquare']].values

In [450]:
data.loc[(data['KitchenSquare'] < 10) & (data['Rooms'] == 3), 'KitchenSquare'] =\
data.loc[data['Rooms'] == 3, 'KitchenSquare'].mean()

data.loc[(data['KitchenSquare'] < 7) & (data['Rooms'] == 2), 'KitchenSquare'] =\
data.loc[data['Rooms'] == 2, 'KitchenSquare'].mean()

data.loc[(data['KitchenSquare'] < 5) & (data['Rooms'] == 1), 'KitchenSquare'] =\
data.loc[data['Rooms'] == 1, 'KitchenSquare'].mean()

In [451]:
data['Square_2'] = data['Square'] ** 2

In [452]:
data.loc[data['Square'] > 500, 'Square'] = data.loc[data['Rooms'] == 2, 'Square'].mean()

In [453]:
data.loc[data['KitchenSquare'] > 500, 'KitchenSquare'] = data.loc[data['Rooms'] == 1, 'KitchenSquare'].mean()

In [454]:
data.loc[data['Square'] < data['LifeSquare'], ['Square', 'LifeSquare']] = \
data.loc[data['Square'] < data['LifeSquare'], ['LifeSquare', 'Square']]

In [455]:
data.loc[(data['KitchenSquare'] < 20) & (data['Rooms'] == 4), 'KitchenSquare'] = \
data.loc[(data['KitchenSquare'] < 20) & (data['Rooms'] == 4), 'KitchenSquare'].mean()

In [456]:
data.loc[(data['KitchenSquare'] < 20) & (data['Rooms'] == 5), 'KitchenSquare'] = \
data.loc[(data['KitchenSquare'] < 20) & (data['Rooms'] == 5), 'KitchenSquare'].mean()

In [457]:
data = pd.get_dummies(data)

In [458]:
train, valid = train_test_split(data, test_size=0.3, random_state=42)

In [459]:
train.shape, valid.shape

((6999, 26), (3000, 26))

In [460]:
district_stat = train.groupby(['DistrictId', 'Rooms'])[['Price']].mean().reset_index().rename(columns={'Price' : 'mean_price'})
district_stat.shape

(623, 3)

In [461]:
train = pd.merge(train, district_stat, on=['DistrictId', 'Rooms'], how='left')
train['mean_price'].isnull().sum()

0

In [462]:
valid = pd.merge(valid, district_stat, on=['DistrictId', 'Rooms'], how='left')
valid['mean_price'].isnull().sum()

51

In [463]:
rooms_stat = train.groupby(['Rooms'])[['Price']].mean().reset_index().rename(columns={'Price':'mean_price2'})

In [464]:
train = pd.merge(train, rooms_stat, on=['Rooms'], how='left')

In [465]:
valid = pd.merge(valid, rooms_stat, on=['Rooms'], how='left')

In [466]:
valid['mean_price2'].isnull().sum()

0

In [467]:
valid['mean_price'].fillna(valid['mean_price2'], inplace=True)

In [468]:
# линейная регрессия

In [469]:
lr = LinearRegression()
features_for_predict = ['Square', 'Square_2','LifeSquare', 'Rooms', 
       'Floor', 'HouseYear', 'Ecology_1',
        'Healthcare_1',
       'Social_1', 'Social_2', 'Ecology_2_A', 'Ecology_2_B',
        'Ecology_3_A', 'Ecology_3_B',
       'Shops_1', 'Shops_2_A',
       'Shops_2_B', 'mean_price',
      ]


In [470]:
lr.fit(train.loc[:, features_for_predict], train['Price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [471]:
pred_train = lr.predict(train.loc[:, features_for_predict])

In [472]:
r2_score(train['Price'], pred_train)

0.7876013191126413

In [473]:
pred = lr.predict(valid.loc[:, features_for_predict])

In [474]:
valid_pred = pd.DataFrame({'valid_price' : valid['Price'], 'pred_valid_price' : pred})
valid_pred.head(10)

,valid_price,pred_valid_price
0,187042.020422,175307.020840
1,215221.034144,179596.619566
2,289085.336309,276734.618216
3,102666.191762,130957.500737
4,158287.271791,141873.070365
5,102577.541221,205238.461235
6,215221.034144,217850.651254
7,215221.034144,214278.446909
8,148458.941020,145997.826861
9,184036.005146,193239.501527


In [475]:
r2_score(valid['Price'], pred)

0.6690633324043544

#### RF

In [476]:
# random forest
from sklearn.ensemble import RandomForestRegressor as RF

In [477]:
rf = RF(max_depth=8, n_estimators=49)

In [478]:
rf.fit(train.loc[:, features_for_predict], train['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=49, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [479]:
pred = rf.predict(train.loc[:, features_for_predict])

In [480]:
r2_score(train['Price'], pred)

0.8605772645565204

In [481]:
pred_valid = rf.predict(valid.loc[:, features_for_predict])

In [482]:
r2_score(valid['Price'], pred_valid)

0.7070732481205463

In [483]:
# for i in range(1, 100, 10):
#     print(i)
#     rf = RF(max_depth=8, n_estimators=20, min_samples_leaf=i)
#     rf.fit(train.loc[:, features_for_predict], train['Price'])
#     pred = rf.predict(train.loc[:, features_for_predict])
#     print(r2_score(train['Price'], pred))
#     pred_valid = rf.predict(valid.loc[:, features_for_predict])
#     print(r2_score(valid['Price'], pred_valid))
#     print()

### предсказание на тесте

In [484]:
test = pd.read_csv('test.csv')

In [485]:
test['LifeSquare'] = test['LifeSquare'].fillna(test['LifeSquare'].mean())
test['Healthcare_1'] = test['Healthcare_1'].fillna(test['Healthcare_1'].mean())

In [486]:
test = pd.get_dummies(test)

In [487]:
test['Square_2'] = test['Square'] ** 2

In [488]:
test = pd.merge(test, district_stat, on=['DistrictId', 'Rooms'], how='left')

In [489]:
test = pd.get_dummies(test)

In [490]:
test['mean_price'].isnull().sum()

100

In [498]:
test['mean_price'].fillna(test['mean_price'].mean(), inplace=True)

In [501]:
test_pred = rf.predict(test.loc[:, features_for_predict])

In [502]:
test['Price'] = test_pred

In [503]:
test.loc[:, ['Id', 'Price']].to_csv('VBurtcev_predictions.csv')